# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    <!--You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
-->
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with 40K kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercises</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>


<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

In [69]:
import pandas as pd

names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Change to KVR1000.csv.gz if this becomes too slow for you
# kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz', 
                   compression='gzip', sep='\t', 
                   index_col=0, names=names,
                   ) 

for kolom in names[1:]:
    kvrdf[kolom]= kvrdf[kolom].astype(str)
print(kvrdf.shape)
kvrdf.head()



(1000, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat


In [70]:
from nltk.corpus import stopwords
import nltk
from collections import Counter
import itertools
from math import log

DutchStop= stopwords.words('dutch')
allvragen= '\n'.join(list(kvrdf.titel))
classes = list(set(list(kvrdf.ministerie)))

Words are treated as lowercase & stopwords are filtered out below

In [71]:
# Term frequencies per word

def strip_string(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: list of all tokens extracted from the string (lowercasing, stopwords, alpha)
    """
    return [w for w in nltk.word_tokenize(string.lower()) if w.isalpha() and not w in set(DutchStop)]
    
def str_to_tf(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: dictionary of all term frequencies: occurance of term in string
    """
    return Counter(strip_string(string))

In [72]:
def str_to_tk(string):
    """ returns token count in a string
    """
    return len(strip_string(string))

## 1. 
Normalize the values for "ministerie" and choose 10 ministeries to work with.

In [89]:
# Only return classes that are about a single ministerie.
def determine_classes(classes):
    """ Remove any class that spans multiple ministeries
    
    :param classes: All classes that occur in kamervragen
    :return:
        norm_classes, a subset of classes with only the classes that span a single ministerie.
    """

    norm_classes = set()
    
    for c in classes:
        add = True

        if c == 'nan':
            continue

        for nc in norm_classes:
            if nc in c:
                add = False
                break
            elif c in nc:
                norm_classes.remove(nc)
                break

        if add:
            norm_classes.add(c)
            
    return norm_classes


# Normalize class c by replacing strange e's with a normal e
# and removing anything between parenthesis.
def normalize_class(c):
    """ Normalize class c by replacing different representations of the e by a normal e
    
    :param c: str, a single class name
    :return:
        str, a single class name with 'normal' e's and no trailing whitespace.
    """

    nc = ""
    parenthesis = False
    
    for char in c:
        if char == '(':
            parenthesis = True
            
        elif char == 'ë':
            char = 'e'
        elif char == 'Ã':
            char = 'e'
        elif char == '«':
            char = ''
            
        if not parenthesis:
            nc += char
            
        if char == ')':
            parenthesis = False
    
    return nc.strip()


def choose_10_classes(class_rows, norm_classes):
    """ Only return the 10 most occuring classes.
    
    :param class_rows: dict, a dictionary mapping a class name to all rows that class occurs in.
    :param norm_classes: set, all normalized class names of classes that only span a single ministerie.
    
    :return:
        set, the normalized names of the 10 most occuring classes
    """
    count = Counter()
    for key, rows in class_rows.items():
        key = normalize_class(key)
        
        if key in norm_classes:
            count[key] += len(rows)
            
    return set([name for name, _ in count.most_common(10)])


def kvrdf_to_10_classes(kvrdf, norm_classes):
    """ Return a copy of kvrdf with normalized class names and only containing rows with classes in norm_classes
    
    :param kvrdf: pd.Dataframe, containing all downloaded information: questions, ministerie, answers, etc.
    :param norm_classes: set, containing all normalized class names of the 10 most occuring classes.
    
    :return:
        pd.Dataframe, kvrdf but with all rows removed that don't contain any of the ministeries in norm_classes.
    """   
    all_classes = set(kvrdf.ministerie)
    
    for i, c in enumerate(all_classes):
        nc = normalize_class(c)
        
        if not (nc in norm_classes):
            kvrdf = kvrdf[kvrdf.ministerie != c]
            
        else:
            kvrdf.loc[kvrdf["ministerie"] == c, "ministerie"] = nc
            
    return kvrdf

# classes is a set of strings of all classes that are in kvrdf
classes = set(kvrdf.ministerie)

# norm classes is a set of strings of classes that only span a single ministerie with normalized names.
norm_classes = {normalize_class(c) for c in determine_classes(classes)}

# class_rows_full is a dictionary mapping a class name to all rows that contain that class.
class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

# classes is adjusted so it now is a set that contains the normalized strings of the 10 most occuring classes.
classes = choose_10_classes(class_rows_full, norm_classes)
print(classes)

# kvrdf is adjusted so any rows that don't contain any class in 'classes' are removed.
# also, all class names are normalized.
kvrdf = kvrdf_to_10_classes(kvrdf, classes)
kvrdf.head()

{'Justitie', 'Verkeer en Waterstaat', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Defensie', 'Landbouw, Natuurbeheer en Visserij', 'Onderwijs, Cultuur en Wetenschappen', 'Economische Zaken', 'Buitenlandse Zaken', 'Sociale Zaken en Werkgelegenheid', 'Volksgezondheid, Welzijn en Sport'}


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat
KVR10004.xml,1999,D66,Vragen naar aanleiding van het bericht in de ...,Kent u het bericht als zou het RIVM onjuiste ...,Ja. Nee. Als reactie op het Volkskrant-artike...,Verkeer en Waterstaat



Classes are extracted from the ministerie column in the dataframe. Any classes that don't occur often or span multiple ministeries, are removed from the dataframe. All other classes have their names normalized.

The normalization is done by changing all variants of the 'e' in a class to a normal 'e'. Also any class that is named 'nan' were removed. And finally, any starting or trailing whitespace were removed from the class name.

## 2.
Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies. It might be easier to use the representation and formula given in MRS section 13.4.1.

In [74]:
#document frequencies per class per term
def TrainMultinomialNB(pd_df, classes):
    """ more reference from p258 onwards of MRS
    
    :param classes: classes which are taken into account for training
    :param pd_df: kamervragen dataframe
    :return:
        V: set, of terms which which are used for classification
        class_priors: dictionary, prior probabilities for each class: P(c) prior[class]
        cond_prob: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
        dfdict: dictionary with document frequencies
        classes: list, classes actually used, temporary until class input is synchronised with pandas dataframe
    """
    class_frequency = Counter([c for c in pd_df.ministerie])
    
    
    # P(c)
    class_priors = {c:class_frequency[c]/sum([class_frequency[cn] for cn in classes]) for c in classes}

    # document frequencies per term
    k = [list(set(strip_string(t))) for t in kvrdf.titel]
    dfdict = Counter(list(itertools.chain.from_iterable(k)))

    #vocabulary
    V = set(dfdict.keys())
    
    # complete rows per class
    class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

    # term frequency per class
    class_tf = {c:str_to_tf('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}
    
    # token count per class (for sum over Tct' in (t' in V), (p259, formula 13.6)
    class_tk = {c:str_to_tk('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}

    #P(t|c)
    cond_prob = {t:{c:(class_tf[c][t] + 1)/(class_tk[c] + 1) for c in classes} for t in V}
    return V, class_priors, cond_prob, dfdict, classes

def filter_vocab(W, V):
    """ 
    
    :param W: list; stripped query
    :param V: set; of vocab used for classification (determined by one or anoter method)
    :return: a list of words that occur in the used vocabulary
    """
    return [w for w in W if w in V]

def ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None):
    """ Applies a Naive Bayes classifier on previously derived priors & conditionals on a query d

    :param classes: list, optional ministeries to classify between 
    :param Vocab: set, of terms which which are used for classification
    :param priors: dictionary, prior probabilities for each class: P(c) prior[class]
    :param conditionals: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
    :param d: string, query
    :return: best classification for query
    """
    W = strip_string(d)
    if Vocab:
        W = filter_vocab(W, Vocab)
    score = {c:log(priors[c]) for c in classes}
    for k in score.keys():
        score[k] += sum([log(conditionals[t][k]) for t in W])    
    best_class = sorted(score, key = lambda key: score[key])[0]
    print(sorted(score, key = lambda key: score[key]))
    return best_class



## 3.
On this collection, train NB text classifiers for 10 different classes with enough and interesting data.

In [75]:
V, class_priors, cond_prob, dfdict, classes = TrainMultinomialNB(kvrdf, classes)
ApplyMultinomialNB(classes, class_priors, cond_prob, "vragen")


['Economische Zaken', 'Defensie', 'Landbouw, Natuurbeheer en Visserij', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Onderwijs, Cultuur en Wetenschappen', 'Sociale Zaken en Werkgelegenheid', 'Buitenlandse Zaken', 'Verkeer en Waterstaat', 'Volksgezondheid, Welzijn en Sport', 'Justitie']


'Economische Zaken'

## 4.
Compute for each term and each of your 10 classes its utility for that class using mutual information.

In [76]:
def extract_class_df(pddf, classes):
    class_dfdict = {c:{} for c in classes}

    for i, c in enumerate(classes):
        for d in kvrdf[kvrdf.ministerie == c].titel:
            for elem in set(strip_string(d)):
                if class_dfdict[c].get(elem, 0):
                    class_dfdict[c][elem] += 1
                else:
                    class_dfdict[c][elem] = 1
    return class_dfdict

cddf = extract_class_df(kvrdf, classes)

In [77]:
def MI(doc_freq, class_doc_freq):
    """ creates a mutual information dictionary for each term, class pair
    
    :param doc_freq: dictionary, contains documents frequencies per term, doc_freq[term]
    :param class_doc_freq: dictionary, contains documents frequencies per term per class, cdf[class][term]
    :return MIdict: dictionary, contains mutual information for each class/term pair, MI[class][term]
    """
    N = sum(doc_freq.values())
    MIdict = {}
    for c in class_doc_freq.keys():
        MIdict[c] = {}
        for t in class_doc_freq[c].keys():
            N11 = class_doc_freq[c][t]
            N10 = doc_freq[t] - N11
            N01 = sum(class_doc_freq[c].values()) - class_doc_freq[c][t]
            N00 = N - doc_freq[t] - (sum(class_doc_freq[c].values()) - class_doc_freq[c][t])
            N1 = N11 + N10
            N0 = N01 + N00
            if N11:
                MIdict[c][t] = N11/N * log( (N*N11)/(N1*N1), 2)
            if N01:
                MIdict[c][t] += N01/N * log( (N*N01)/(N0*N1) , 2)
            if N10:
                MIdict[c][t] += N10/N * log( (N*N10)/(N1*N0) , 2)
            if N00:
                MIdict[c][t] += N00/N * log(  (N*N00)/(N0*N0), 2)
    return MIdict

MIdict = MI(dfdict, cddf)

## 5.
For each class, show the top 10 words as in Figure 13.7 in MRS.

In [78]:
def top_MI(MIdict, N):
    """ 
    
    :param MIdict: a dictionary with mutual information for each MIdict[class][term]
    :param N: top N MI terms returned per class
    :return: a dictionary of the top N MI predictors of each class    
    """
    resdict = {}
    for c in MIdict.keys():
        resdict[c] = sorted(MIdict[c].items(), key = lambda key: -MIdict[c][key[0]])[:N]
    return resdict

def top_MI_to_markdown(topMI):
    firstline = classes_table_markdown(topMI)
    rest_of_table = values_table_markdown(topMI)
    return firstline + rest_of_table


def classes_table_markdown(topmidict):
    return "| []() | " + " | []() | ".join(topmidict.keys()) + "|\n" + "|" + " | ".join(['----- | -----' for _ in topmidict.keys()]) + "| \n"

def values_table_markdown(topmidict):
    returnstring = ""

    for i, elem in enumerate(topmidict[list(topmidict.keys())[0]]):
        returnstring += " | " + " | ".join([pair_to_markdown(topmidict[c][i]) for c in topmidict.keys()]) + " | \n"
    return returnstring
            
def pair_to_markdown(pair):
    return pair[0] + " | " + str(pair[1])


def topMI_to_vocab(topmidict):
    vocab_l_o_l = [[elem[0] for elem in topmidict[c]] for c in topmidict.keys()]
    return set(list(itertools.chain.from_iterable(vocab_l_o_l)))



In [79]:
topfeatures = top_MI(MIdict, 1)

dfdict2 = {t:v for t,v in dfdict.items() if v > 5}
cddf2 = {c:{t:cddf[c][t] for t in cddf[c].keys() if t in dfdict2.keys()} for c in cddf}
MIdict2 = MI(dfdict2, cddf2)
topfeatures2 = top_MI(MIdict, 85)


In [80]:
from IPython.display import display, Markdown, Latex


In [81]:
topfeatures_MD = top_MI_to_markdown(topfeatures)
topfeatures_MD2 = top_MI_to_markdown(topfeatures2)

# bug is because of empty classes & not being able to recognize classes in normalized form

display(Markdown(topfeatures_MD2))

| []() | Justitie | []() | Verkeer en Waterstaat | []() | Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer | []() | Defensie | []() | Landbouw, Natuurbeheer en Visserij | []() | Onderwijs, Cultuur en Wetenschappen | []() | Economische Zaken | []() | Buitenlandse Zaken | []() | Sociale Zaken en Werkgelegenheid | []() | Volksgezondheid, Welzijn en Sport|
|----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | -----| 
 | blijt | 1.930452001280508 | geluidsmetingen | 0.9064642091914905 | toevoeging | 0.7415464892478193 | fosforgranaten | 0.30255954304494675 | subsidiëring | 0.3896423179577661 | onvrijwillige | 0.7542116195739694 | proefboringen | 0.4346903879604562 | anderzijds | 1.0017786861976063 | verkeert | 0.9046081268620453 | zorginstellingen | 1.5588902321464466 | 
 | professioneel | 1.930452001280508 | onbetrouwbaarheid | 0.9064642091914905 | nutssectoren | 0.7415464892478193 | haveningang | 0.30255954304494675 | haalbaarheidsstudie | 0.3896423179577661 | enige | 0.7542116195739694 | clyde | 0.4346903879604562 | ambassadeur | 1.0017786861976063 | achtergelaten | 0.9046081268620453 | verpleegkundigen | 1.5588902321464466 | 
 | kliniek | 1.930452001280508 | filemeldingen | 0.9064642091914905 | voetnoten | 0.7415464892478193 | vlak | 0.30255954304494675 | project | 0.3896423179577661 | onderwijsinspecteurs | 0.7542116195739694 | petroleum | 0.4346903879604562 | vrijheden | 1.0017786861976063 | nadelige | 0.9046081268620453 | traumatische | 1.5588902321464466 | 
 | twijfels | 1.930452001280508 | gehanteerd | 0.9064642091914905 | handen | 0.7415464892478193 | oosterschelde | 0.30255954304494675 | envirodung | 0.3896423179577661 | inschatting | 0.7542116195739694 | exploratie | 0.4346903879604562 | debat | 1.0017786861976063 | werkplek | 0.9046081268620453 | veldhuis | 1.5588902321464466 | 
 | vervolgd | 1.930452001280508 | onjuiste | 0.9064642091914905 | liberaal | 0.7415464892478193 | bodem | 0.30255954304494675 | dioxinevrij | 0.3896423179577661 | ibo | 0.7542116195739694 | consortium | 0.4346903879604562 | balans | 1.0017786861976063 | arbeidsgehandicapte | 0.9046081268620453 | vechten | 1.5588902321464466 | 
 | psychiatrische | 1.930452001280508 | gepresenteerd | 0.9064642091914905 | grondeigenaar | 0.7415464892478193 | ton | 0.30255954304494675 | weyl | 0.3896423179577661 | organisation | 0.7542116195739694 | hanze | 0.4346903879604562 | integriteit | 1.0017786861976063 | lisv | 0.9046081268620453 | waarneming | 1.5588902321464466 | 
 | patient | 1.930452001280508 | milieuaspecten | 0.9064642091914905 | voorkeursrecht | 0.7415464892478193 | zierikzee | 0.30255954304494675 | beef | 0.3896423179577661 | baccalaureate | 0.7542116195739694 | koreaans | 0.4346903879604562 | respect | 1.0017786861976063 | loonkostensubsidie | 0.9046081268620453 | eilander | 1.5588902321464466 | 
 | persoonlijke | 1.930452001280508 | bentoniet | 0.9064642091914905 | utrechtse | 0.7415464892478193 | przzn | 0.30255954304494675 | products | 0.3896423179577661 | vacature | 0.7542116195739694 | gunnen | 0.4346903879604562 | souvereiniteit | 1.0017786861976063 | hogere | 0.9046081268620453 | betalingsysteem | 1.5588902321464466 | 
 | betaalgedrag | 1.930452001280508 | boortechniek | 0.9064642091914905 | projectontwikkelaar | 0.7415464892478193 | benutten | 0.30255954304494675 | firma | 0.3896423179577661 | ondersteunende | 0.7542116195739694 | platform | 0.4346903879604562 | territoriale | 1.0017786861976063 | reistijden | 0.9046081268620453 | overeenkomsten | 1.5588902321464466 | 
 | denkt | 1.930452001280508 | bestemming | 0.9064642091914905 | milieuhygiene | 0.7415464892478193 | proef | 0.30255954304494675 | expliciete | 0.3896423179577661 | krantenberichten | 0.7542116195739694 | continentaal | 0.4346903879604562 | veiligheidsraad | 1.0017786861976063 | gunning | 0.9046081268620453 | declareert | 1.5588902321464466 | 
 | omkoping | 1.930452001280508 | resten | 0.9064642091914905 | doorgaat | 0.7415464892478193 | geworven | 0.30255954304494675 | exportproblemen | 0.3896423179577661 | stagebemiddeling | 0.7542116195739694 | plat | 0.4346903879604562 | walsum | 1.0017786861976063 | gehandicaptenvervoer | 0.9046081268620453 | verrichting | 1.5588902321464466 | 
 | geldstromen | 1.930452001280508 | tunnels | 0.9064642091914905 | teelt | 0.7415464892478193 | vijfhonderd | 0.30255954304494675 | grootschalig | 0.3896423179577661 | onderwijstaak | 0.7542116195739694 | produktieplatform | 0.4346903879604562 | mensenrechtenactivisten | 1.0017786861976063 | beschikbaarheid | 0.9046081268620453 | aflopen | 1.5588902321464466 | 
 | grip | 1.930452001280508 | boren | 0.9064642091914905 | gemanipuleerde | 0.7415464892478193 | soldaten | 0.30255954304494675 | dioxinecrisis | 0.3896423179577661 | normale | 0.7542116195739694 | energietechniek | 0.4346903879604562 | oppakken | 1.0017786861976063 | langere | 0.9046081268620453 | specialist | 1.5588902321464466 | 
 | smeergeldaffaires | 1.930452001280508 | vervuild | 0.9064642091914905 | aardappelen | 0.7415464892478193 | clusterbommen | 0.30255954304494675 | voorbeeld | 0.3896423179577661 | middelbaar | 0.7542116195739694 | dynaf | 0.4346903879604562 | oezbekistan | 1.0017786861976063 | fin | 0.9046081268620453 | definitie | 1.5588902321464466 | 
 | verjaard | 1.930452001280508 | verdubbeling | 0.9064642091914905 | uitblijven | 0.7415464892478193 | slachtoffer | 0.30255954304494675 | voedselketen | 0.3896423179577661 | wbo | 0.7542116195739694 | strategische | 0.4346903879604562 | aanhef | 1.0017786861976063 | zalm | 0.9046081268620453 | recept | 1.5588902321464466 | 
 | bezoekers | 1.930452001280508 | traject | 0.9064642091914905 | agrarisch | 0.7415464892478193 | daarbij | 0.30255954304494675 | pluimveehouders | 0.3896423179577661 | oetc | 0.7542116195739694 | vergunningenstop | 0.4346903879604562 | ontvlucht | 1.0017786861976063 | arbeidsongeschiktheidsuitkering | 0.9046081268620453 | zelfzorgmedicijnen | 1.5588902321464466 | 
 | maat | 1.930452001280508 | landelijk | 0.9064642091914905 | vonnis | 0.7415464892478193 | resp | 0.30255954304494675 | optredende | 0.3896423179577661 | formatiebesluit | 0.7542116195739694 | goederen | 0.4346903879604562 | serviers | 1.0017786861976063 | sollicitatietrainingen | 0.9046081268620453 | afleveren | 1.5588902321464466 | 
 | gecontroleerd | 1.930452001280508 | vaargeulen | 0.9064642091914905 | zendinstallaties | 0.7415464892478193 | missie | 0.30255954304494675 | dierziekten | 0.3896423179577661 | decentralisatie | 0.7542116195739694 | stroomaggregaten | 0.4346903879604562 | velen | 1.0017786861976063 | tilburg | 0.9046081268620453 | drogisten | 1.5588902321464466 | 
 | uitgaansgelegenheden | 1.930452001280508 | hollandse | 0.9064642091914905 | haarlemmermeer | 0.7415464892478193 | militairen | 0.30255954304494675 | omvangrijke | 0.3896423179577661 | tegenstelling | 0.7542116195739694 | defensieapparatuur | 0.4346903879604562 | zigeuners | 1.0017786861976063 | vacatures | 0.9046081268620453 | uiteindelijke | 1.5588902321464466 | 
 | permanent | 1.930452001280508 | baggerspecie | 0.9064642091914905 | libertel | 0.7415464892478193 | detectie | 0.30255954304494675 | havenslib | 0.3896423179577661 | basisonderwijs | 0.7542116195739694 | oosterveen | 0.4346903879604562 | procureur | 1.0017786861976063 | omscholingen | 0.9046081268620453 | geneesmiddelen | 1.5588902321464466 | 
 | disco | 1.930452001280508 | ijssel | 0.9064642091914905 | benw | 0.7415464892478193 | opruiming | 0.30255954304494675 | begrazing | 0.3896423179577661 | bestrijden | 0.7542116195739694 | verwerkt | 0.4346903879604562 | pais | 1.0017786861976063 | tuinbouw | 0.9046081268620453 | geneesmiddelenvoorziening | 1.5588902321464466 | 
 | anja | 1.930452001280508 | berging | 0.9064642091914905 | milieuvriendelijke | 0.7415464892478193 | antipersoneelsmijnen | 0.30255954304494675 | verontreinigd | 0.3896423179577661 | lerarentekort | 0.7542116195739694 | inro | 0.4346903879604562 | uitleveringsverzoek | 1.0017786861976063 | poolse | 0.9046081268620453 | opdraaien | 1.5588902321464466 | 
 | wegenverkeerswet | 1.930452001280508 | verontreinigde | 0.9064642091914905 | sloop | 0.7415464892478193 | stress | 0.30255954304494675 | standpunt | 0.3896423179577661 | terugdringen | 0.7542116195739694 | gemeentelijke | 0.4346903879604562 | el | 1.0017786861976063 | prof | 0.9046081268620453 | ru | 1.5588902321464466 | 
 | arresten | 1.930452001280508 | uitbaggeren | 0.9064642091914905 | verwijderingsheffing | 0.7415464892478193 | verzorging | 0.30255954304494675 | opgespoten | 0.3896423179577661 | herintreders | 0.7542116195739694 | heffingsgrondslagen | 0.4346903879604562 | annuleren | 1.0017786861976063 | scholing | 0.9046081268620453 | abortuspil | 1.5588902321464466 | 
 | kellenaers | 1.930452001280508 | attracties | 0.9064642091914905 | mindere | 0.7415464892478193 | ingestoken | 0.30255954304494675 | veterinaire | 0.3896423179577661 | oormerken | 0.7542116195739694 | bevat | 0.4346903879604562 | zichtbaarder | 1.0017786861976063 | bedrijfstakgewijze | 0.9046081268620453 | mifygene | 1.5588902321464466 | 
 | waarnemers | 1.930452001280508 | toeristische | 0.9064642091914905 | milieuwinst | 0.7415464892478193 | wervingscampagne | 0.30255954304494675 | biodiesel | 0.3896423179577661 | lambrechts | 0.7542116195739694 | toeristenbelasting | 0.4346903879604562 | politiegeweld | 1.0017786861976063 | kbs | 0.9046081268620453 | hygienisch | 1.5588902321464466 | 
 | arrestatie | 1.930452001280508 | pretparken | 0.9064642091914905 | leiding | 0.7415464892478193 | functionarissen | 0.30255954304494675 | braakliggende | 0.3896423179577661 | kapitaalkrachtige | 0.7542116195739694 | opbrengsten | 0.4346903879604562 | rechtsvervolging | 1.0017786861976063 | kra | 0.9046081268620453 | tatoeeerders | 1.5588902321464466 | 
 | alarmteam | 1.930452001280508 | bereikbaarheid | 0.9064642091914905 | kwaliteit | 0.7415464892478193 | reductie | 0.30255954304494675 | koolzaad | 0.3896423179577661 | enertel | 0.7542116195739694 | standaardisatie | 0.4346903879604562 | overwegen | 1.0017786861976063 | arbeidsinpassing | 0.9046081268620453 | piercers | 1.5588902321464466 | 
 | protestactie | 1.930452001280508 | goederenspoorlijn | 0.9064642091914905 | huishoudwater | 0.7415464892478193 | bewaking | 0.30255954304494675 | geteeld | 0.3896423179577661 | buitenkernen | 0.7542116195739694 | splitsing | 0.4346903879604562 | chileense | 1.0017786861976063 | genomen | 0.9046081268620453 | steriel | 1.5588902321464466 | 
 | gronden | 1.930452001280508 | vervoersvoorzieningen | 0.9064642091914905 | grijs | 0.7415464892478193 | uittocht | 0.30255954304494675 | varkensbedrijven | 0.3896423179577661 | opheffingsnorm | 0.7542116195739694 | steunkaart | 0.4346903879604562 | stabiliseringsproces | 1.0017786861976063 | kaderregeling | 0.9046081268620453 | trekkingsrechten | 1.5588902321464466 | 
 | helwijk | 1.930452001280508 | toegankelijkheid | 0.9064642091914905 | alternatieven | 0.7415464892478193 | zwaar | 0.30255954304494675 | beoordeling | 0.3896423179577661 | kamperveen | 0.7542116195739694 | steunmaatregelen | 0.4346903879604562 | stad | 1.0017786861976063 | nader | 0.9046081268620453 | uitvoeringsregels | 1.5588902321464466 | 
 | humanitaire | 1.930452001280508 | handicap | 0.9064642091914905 | chloordosering | 0.7415464892478193 | druk | 0.30255954304494675 | voodat | 0.3896423179577661 | voortbestaan | 0.7542116195739694 | moeizaam | 0.4346903879604562 | buitenproportionele | 1.0017786861976063 | circulaire | 0.9046081268620453 | persoonsgebonden | 1.5588902321464466 | 
 | verblijfsvergunning | 1.930452001280508 | zestien | 0.9064642091914905 | zwemwater | 0.7415464892478193 | grensbewaking | 0.30255954304494675 | onmisbaarheid | 0.3896423179577661 | zuideinde | 0.7542116195739694 | verlopen | 0.4346903879604562 | grozni | 1.0017786861976063 | vries | 0.9046081268620453 | verandering | 1.5588902321464466 | 
 | alsnog | 1.930452001280508 | reisservice | 0.9064642091914905 | milieuhygiëne | 0.7415464892478193 | massale | 0.30255954304494675 | gespreksronde | 0.3896423179577661 | optrekken | 0.7542116195739694 | hiermee | 0.4346903879604562 | hulpfonds | 1.0017786861976063 | bijzondere | 0.9046081268620453 | pgb | 1.5588902321464466 | 
 | genoeg | 1.930452001280508 | oedayraj | 0.9064642091914905 | boorlocaties | 0.7415464892478193 | vreemdelingentoezicht | 0.30255954304494675 | gewasbescherming | 0.3896423179577661 | ijsselmuiden | 0.7542116195739694 | concerns | 0.4346903879604562 | azië | 1.0017786861976063 | weten | 0.9046081268620453 | drang | 1.5588902321464466 | 
 | korthals | 1.930452001280508 | singh | 0.9064642091914905 | goedkeuring | 0.7415464892478193 | landmachtpersoneel | 0.30255954304494675 | pine | 0.3896423179577661 | vermogen | 0.7542116195739694 | multinationale | 0.4346903879604562 | ecip | 1.0017786861976063 | bijstand | 0.9046081268620453 | lozing | 1.5588902321464466 | 
 | banden | 1.930452001280508 | giskes | 0.9064642091914905 | gs | 0.7415464892478193 | feddema | 0.30255954304494675 | company | 0.3896423179577661 | cash | 0.7542116195739694 | accijnzen | 0.4346903879604562 | meningsverschillen | 1.0017786861976063 | afschrift | 0.9046081268620453 | otterloo | 1.5588902321464466 | 
 | smartshops | 1.930452001280508 | varma | 0.9064642091914905 | buitengebied | 0.7415464892478193 | hr | 0.30255954304494675 | exploitant | 0.3896423179577661 | introductieweken | 0.7542116195739694 | groeiende | 0.4346903879604562 | clark | 1.0017786861976063 | ns | 0.9046081268620453 | zachte | 1.5588902321464466 | 
 | gedogen | 1.930452001280508 | vrachttoestel | 0.9064642091914905 | eemsmond | 0.7415464892478193 | rapporten | 0.30255954304494675 | natuurvriendenhuizen | 0.3896423179577661 | gepropageerd | 0.7542116195739694 | geheven | 0.4346903879604562 | vredesmacht | 1.0017786861976063 | verzorgen | 0.9046081268620453 | toegelaten | 1.5588902321464466 | 
 | coffeeshop | 1.930452001280508 | overvliegend | 0.9064642091914905 | onthouden | 0.7415464892478193 | tribune | 0.30255954304494675 | natuurwaarden | 0.3896423179577661 | gegeven | 0.7542116195739694 | bestelling | 0.4346903879604562 | vervroegd | 1.0017786861976063 | twintig | 0.9046081268620453 | beschermd | 1.5588902321464466 | 
 | zwijndrecht | 1.930452001280508 | autoloze | 0.9064642091914905 | gedeputeerde | 0.7415464892478193 | watergate | 0.30255954304494675 | duinen | 0.3896423179577661 | verkeersles | 0.7542116195739694 | persoon | 0.4346903879604562 | aftreden | 1.0017786861976063 | ambulances | 0.9046081268620453 | ribw | 1.5588902321464466 | 
 | uitzettingsbeleid | 1.930452001280508 | zondag | 0.9064642091914905 | tc | 0.7415464892478193 | aanhoudend | 0.30255954304494675 | konijnen | 0.3896423179577661 | fondsen | 0.7542116195739694 | wettig | 0.4346903879604562 | april | 1.0017786861976063 | evaluatie | 0.9046081268620453 | dakloze | 1.5588902321464466 | 
 | legaliseren | 1.930452001280508 | toiletvoorzieningen | 0.9064642091914905 | megafusie | 0.7415464892478193 | brandweerpersoneel | 0.30255954304494675 | voedsel | 0.3896423179577661 | cruciale | 0.7542116195739694 | slijtersbedrijf | 0.4346903879604562 | michael | 1.0017786861976063 | bekneld | 0.9046081268620453 | gezinnen | 1.5588902321464466 | 
 | moskou | 1.930452001280508 | milieubelasting | 0.9064642091914905 | waterleidingbedrijven | 0.7415464892478193 | soesterberg | 0.30255954304494675 | bloemendaal | 0.3896423179577661 | bleek | 0.7542116195739694 | gedestilleerd | 0.4346903879604562 | sir | 1.0017786861976063 | werknemer | 0.9046081268620453 | internaten | 1.5588902321464466 | 
 | autodiefstal | 1.930452001280508 | toiletsystemen | 0.9064642091914905 | aquilex | 0.7415464892478193 | volkel | 0.30255954304494675 | hazen | 0.3896423179577661 | kennisgeving | 0.7542116195739694 | maximaal | 0.4346903879604562 | wesley | 1.0017786861976063 | rie | 0.9046081268620453 | schipperskinderen | 1.5588902321464466 | 
 | rechercheurs | 1.930452001280508 | pleziervaartuigen | 0.9064642091914905 | gelderland | 0.7415464892478193 | ernst | 0.30255954304494675 | strandetablissementen | 0.3896423179577661 | fondsenbrief | 0.7542116195739694 | gevestigde | 0.4346903879604562 | jackson | 1.0017786861976063 | betonelementen | 0.9046081268620453 | verhoging | 1.5588902321464466 | 
 | rek | 1.930452001280508 | lozingen | 0.9064642091914905 | loodfabriek | 0.7415464892478193 | asbestvervuiling | 0.30255954304494675 | patatvos | 0.3896423179577661 | meldt | 0.7542116195739694 | flessen | 0.4346903879604562 | dna | 1.0017786861976063 | overleed | 0.9046081268620453 | ouderbijdrage | 1.5588902321464466 | 
 | rechtseenheidskamer | 1.930452001280508 | bevorderen | 0.9064642091914905 | metal | 0.7415464892478193 | gaande | 0.30255954304494675 | trekt | 0.3896423179577661 | ploeg | 0.7542116195739694 | electriciteitscentrales | 0.4346903879604562 | inheemse | 1.0017786861976063 | raakte | 0.9046081268620453 | zwerfjongeren | 1.5588902321464466 | 
 | punten | 1.930452001280508 | korter | 0.9064642091914905 | verbruikte | 0.7415464892478193 | peel | 0.30255954304494675 | vossen | 0.3896423179577661 | leerlingbegeleiding | 0.7542116195739694 | kerncentrale | 0.4346903879604562 | volken | 1.0017786861976063 | betonfabriek | 0.9046081268620453 | ambulancezorg | 1.5588902321464466 | 
 | strijdig | 1.930452001280508 | francisco | 0.9064642091914905 | milieuvoorschriften | 0.7415464892478193 | geleide | 0.30255954304494675 | boulevards | 0.3896423179577661 | nieuws | 0.7542116195739694 | borssele | 0.4346903879604562 | materiaal | 1.0017786861976063 | bedrijfsongeval | 0.9046081268620453 | bemiddelt | 1.5588902321464466 | 
 | meerdere | 1.930452001280508 | klm | 0.9064642091914905 | arbeidsomstandigheden | 0.7415464892478193 | wellicht | 0.30255954304494675 | zandvoort | 0.3896423179577661 | avbb | 0.7542116195739694 | beslissingen | 0.4346903879604562 | annan | 1.0017786861976063 | erkende | 0.9046081268620453 | arbeidsgerelateerde | 1.5588902321464466 | 
 | gevonden | 1.930452001280508 | km | 0.9064642091914905 | overtreden | 0.7415464892478193 | bond | 0.30255954304494675 | vormen | 0.3896423179577661 | instantie | 0.7542116195739694 | recepties | 0.4346903879604562 | sanctiecommissie | 1.0017786861976063 | scholingsinstituut | 0.9046081268620453 | omzeilt | 1.5588902321464466 | 
 | argos | 1.930452001280508 | duur | 0.9064642091914905 | loodaccu | 0.7415464892478193 | abvakabo | 0.30255954304494675 | muizen | 0.3896423179577661 | salaris | 0.7542116195739694 | host | 0.4346903879604562 | kofi | 1.0017786861976063 | eega | 0.9046081268620453 | contact | 1.5588902321464466 | 
 | dierbevrijdingsfront | 1.930452001280508 | retour | 0.9064642091914905 | blanc | 0.7415464892478193 | belastingdienst | 0.30255954304494675 | gesignaleerd | 0.3896423179577661 | elkaars | 0.7542116195739694 | wto | 0.4346903879604562 | henan | 1.0017786861976063 | borne | 0.9046081268620453 | cagin | 1.5588902321464466 | 
 | beursfraudezaken | 1.930452001280508 | afstand | 0.9064642091914905 | verwerking | 0.7415464892478193 | werkzaam | 0.30255954304494675 | weidevogels | 0.3896423179577661 | secundaire | 0.7542116195739694 | bestond | 0.4346903879604562 | staatsveiligheidsdiensten | 1.0017786861976063 | inkomstenformulier | 0.9046081268620453 | tweedelijnsarbozorg | 1.5588902321464466 | 
 | fred | 1.930452001280508 | asfaltbeton | 0.9064642091914905 | spoor | 0.7415464892478193 | orahovac | 0.30255954304494675 | hormoonmaffia | 0.3896423179577661 | betere | 0.7542116195739694 | ontmoeten | 0.4346903879604562 | veiligheidspolitie | 1.0017786861976063 | specificeren | 0.9046081268620453 | aandoeningen | 1.5588902321464466 | 
 | versterking | 1.930452001280508 | gladheidsbestrijding | 0.9064642091914905 | diesellocs | 0.7415464892478193 | vluchtelingen | 0.30255954304494675 | hormoonwetgeving | 0.3896423179577661 | schooljournaal | 0.7542116195739694 | staatshoofden | 0.4346903879604562 | plaatselijke | 1.0017786861976063 | uitkeringsgerechtigde | 0.9046081268620453 | tabaksindustrie | 1.5588902321464466 | 
 | verwijst | 1.930452001280508 | zoab | 0.9064642091914905 | sp | 0.7415464892478193 | konvooi | 0.30255954304494675 | vluchtroute | 0.3896423179577661 | dreigt | 0.7542116195739694 | diners | 0.4346903879604562 | xian | 1.0017786861976063 | wychen | 0.9046081268620453 | toevoegt | 1.5588902321464466 | 
 | cohen | 1.930452001280508 | winterse | 0.9064642091914905 | huurverhoging | 0.7415464892478193 | lobbyist | 0.30255954304494675 | hormoneneenheid | 0.3896423179577661 | vnu | 0.7542116195739694 | delegaties | 0.4346903879604562 | betrokkenheid | 1.0017786861976063 | inleveren | 0.9046081268620453 | veevoer | 1.5588902321464466 | 
 | legalisering | 1.930452001280508 | methode | 0.9064642091914905 | verslag | 0.7415464892478193 | brengt | 0.30255954304494675 | inspectiedienst | 0.3896423179577661 | stoten | 0.7542116195739694 | organization | 0.4346903879604562 | studentendemonstraties | 1.0017786861976063 | dag | 0.9046081268620453 | levensmiddelen | 1.5588902321464466 | 
 | jus | 1.930452001280508 | casestudie | 0.9064642091914905 | staal | 0.7415464892478193 | samenhangt | 0.30255954304494675 | aid | 0.3896423179577661 | verschraling | 0.7542116195739694 | betrouwbare | 0.4346903879604562 | authorities | 1.0017786861976063 | stopzetting | 0.9046081268620453 | controlemaatregelen | 1.5588902321464466 | 
 | plek | 1.930452001280508 | verder | 0.9064642091914905 | basco | 0.7415464892478193 | gevoelige | 0.30255954304494675 | korenwolven | 0.3896423179577661 | ervoor | 0.7542116195739694 | statistieken | 0.4346903879604562 | local | 1.0017786861976063 | patstelling | 0.9046081268620453 | transfersom | 1.5588902321464466 | 
 | mislopen | 1.930452001280508 | veiligheids | 0.9064642091914905 | nv | 0.7415464892478193 | meijling | 0.30255954304494675 | vangen | 0.3896423179577661 | eren | 0.7542116195739694 | encryptiesleutel | 0.4346903879604562 | power | 1.0017786861976063 | bureaus | 0.9046081268620453 | wegkopen | 1.5588902321464466 | 
 | toegewezen | 1.930452001280508 | verkeersleiding | 0.9064642091914905 | principe | 0.7415464892478193 | actief | 0.30255954304494675 | fokprogramma | 0.3896423179577661 | procent | 0.7542116195739694 | nt | 0.4346903879604562 | vacuum | 1.0017786861976063 | banenprojecten | 0.9046081268620453 | gespecialiseerd | 1.5588902321464466 | 
 | opgesloten | 1.930452001280508 | luchthavens | 0.9064642091914905 | waarde | 0.7415464892478193 | gmelich | 0.30255954304494675 | gevangenschap | 0.3896423179577661 | sturen | 0.7542116195739694 | le | 0.4346903879604562 | establishment | 1.0017786861976063 | vakantieduur | 0.9046081268620453 | opname | 1.5588902321464466 | 
 | politiecel | 1.930452001280508 | overheidsbijdragen | 0.9064642091914905 | bankiers | 0.7415464892478193 | apachehelikopters | 0.30255954304494675 | zetten | 0.3896423179577661 | kilometer | 0.7542116195739694 | nsa | 0.4346903879604562 | plaatse | 1.0017786861976063 | verwarring | 0.9046081268620453 | reguliere | 1.5588902321464466 | 
 | scheiden | 1.930452001280508 | vermelden | 0.9064642091914905 | syndicaten | 0.7415464892478193 | opleiding | 0.30255954304494675 | term | 0.3896423179577661 | verderop | 0.7542116195739694 | national | 0.4346903879604562 | training | 1.0017786861976063 | gebruikelijke | 0.9046081268620453 | opgelopen | 1.5588902321464466 | 
 | uiteenlopende | 1.930452001280508 | reisorganisaties | 0.9064642091914905 | woningcorporatie | 0.7415464892478193 | vorming | 0.30255954304494675 | samenleving | 0.3896423179577661 | buurt | 0.7542116195739694 | security | 0.4346903879604562 | benoemen | 1.0017786861976063 | werkgeversfraude | 0.9046081268620453 | medicijnenprijzen | 1.5588902321464466 | 
 | reisroutes | 1.930452001280508 | prijzen | 0.9064642091914905 | ingestemd | 0.7415464892478193 | mobiele | 0.30255954304494675 | kieming | 0.3896423179577661 | voortzetting | 0.7542116195739694 | windows | 0.4346903879604562 | unmik | 1.0017786861976063 | flexwerkers | 0.9046081268620453 | vrijlaten | 1.5588902321464466 | 
 | voordoen | 1.930452001280508 | vliegtickets | 0.9064642091914905 | vastgoed | 0.7415464892478193 | eenheid | 0.30255954304494675 | benaming | 0.3896423179577661 | afneemt | 0.7542116195739694 | agency | 0.4346903879604562 | leadership | 1.0017786861976063 | werkgevers | 0.9046081268620453 | borst | 1.5588902321464466 | 
 | asiel | 1.930452001280508 | vergelijking | 0.9064642091914905 | tegenwoordig | 0.7415464892478193 | pelotons | 0.30255954304494675 | zaad | 0.3896423179577661 | techniek | 0.7542116195739694 | geeft | 0.4346903879604562 | filling | 1.0017786861976063 | arbeidsbureaus | 0.9046081268620453 | koopmans | 1.5588902321464466 | 
 | deurwaarders | 1.930452001280508 | vliegmaatschappijen | 0.9064642091914905 | smeersubsidies | 0.7415464892478193 | inzetbare | 0.30255954304494675 | genetic | 0.3896423179577661 | wetenschapswinkel | 0.7542116195739694 | wereldwijd | 0.4346903879604562 | onmiddellijk | 1.0017786861976063 | boer | 0.9046081268620453 | adviseur | 1.5588902321464466 | 
 | vestiging | 1.930452001280508 | mededingingsautoriteit | 0.9064642091914905 | stem | 0.7415464892478193 | weert | 0.30255954304494675 | use | 0.3896423179577661 | monumentwaardige | 0.7542116195739694 | monde | 0.4346903879604562 | interin | 1.0017786861976063 | slecht | 0.9046081268620453 | verzekerd | 1.5588902321464466 | 
 | gerechtsdeurwaarders | 1.930452001280508 | zandwinprojecten | 0.9064642091914905 | delftse | 0.7415464892478193 | bestemmen | 0.30255954304494675 | wetenschappelijke | 0.3896423179577661 | economie | 0.7542116195739694 | elektriciteitsvoorziening | 0.4346903879604562 | mission | 1.0017786861976063 | ingepast | 0.9046081268620453 | waarvoor | 1.5588902321464466 | 
 | rechtszaak | 1.930452001280508 | zandwinningsbedrijven | 0.9064642091914905 | breau | 0.7415464892478193 | afstoting | 0.30255954304494675 | technologies | 0.3896423179577661 | kerkgebouwen | 0.7542116195739694 | hakvoort | 0.4346903879604562 | municipal | 1.0017786861976063 | ingeschreven | 0.9046081268620453 | wachtlijst | 1.5588902321464466 | 
 | verklaard | 1.930452001280508 | nma | 0.9064642091914905 | focus | 0.7415464892478193 | defence | 0.30255954304494675 | vaak | 0.3896423179577661 | kerkbesturen | 0.7542116195739694 | marktwerking | 0.4346903879604562 | ogoniland | 1.0017786861976063 | avondopleiding | 0.9046081268620453 | huishoudelijke | 1.5588902321464466 | 
 | verstrijken | 1.930452001280508 | geraamd | 0.9064642091914905 | transformatoren | 0.7415464892478193 | siegenland | 0.30255954304494675 | restriction | 0.3896423179577661 | ontheffing | 0.7542116195739694 | managementteam | 0.4346903879604562 | processen | 1.0017786861976063 | doventolk | 0.9046081268620453 | hittegolf | 1.5588902321464466 | 
 | redelijke | 1.930452001280508 | vallen | 0.9064642091914905 | crone | 0.7415464892478193 | vaste | 0.30255954304494675 | gurt | 0.3896423179577661 | krappe | 0.7542116195739694 | toezicht | 0.4346903879604562 | geweldsuitbarstingen | 1.0017786861976063 | gebarentaal | 0.9046081268620453 | oversterfte | 1.5588902321464466 | 
 | bloed | 1.930452001280508 | stijgers | 0.9064642091914905 | verwijderingsbijdrage | 0.7415464892478193 | burgervliegveld | 0.30255954304494675 | terletse | 0.3896423179577661 | systematiek | 0.7542116195739694 | dte | 0.4346903879604562 | daarop | 1.0017786861976063 | dove | 0.9046081268620453 | optreedt | 1.5588902321464466 | 
 | promillagegrens | 1.930452001280508 | julianakanaal | 0.9064642091914905 | bedrijfspanden | 0.7415464892478193 | marinevliegkamp | 0.30255954304494675 | erfpacht | 0.3896423179577661 | ravestein | 0.7542116195739694 | graydon | 0.4346903879604562 | onlangs | 1.0017786861976063 | arbeidsproces | 0.9046081268620453 | maandelijkse | 1.5588902321464466 | 
 | alcohol | 1.930452001280508 | sluizen | 0.9064642091914905 | renovatie | 0.7415464892478193 | keulen | 0.30255954304494675 | metende | 0.3896423179577661 | rijksbijdrage | 0.7542116195739694 | verduidelijking | 0.4346903879604562 | srebrenica | 1.0017786861976063 | halveren | 0.9046081268620453 | ca | 1.5588902321464466 | 
 | camp | 1.930452001280508 | drijvende | 0.9064642091914905 | kpmg | 0.7415464892478193 | vliegen | 0.30255954304494675 | heide | 0.3896423179577661 | btw | 0.7542116195739694 | geconcentreerde | 0.4346903879604562 | gehouden | 1.0017786861976063 | streefcijfer | 0.9046081268620453 | overhouden | 1.5588902321464466 | 
 | oven | 1.930452001280508 | voicemail | 0.9064642091914905 | zonnepanelen | 0.7415464892478193 | oprichtingsceremonie | 0.30255954304494675 | vliegveld | 0.3896423179577661 | compenseren | 0.7542116195739694 | detailhandels | 0.4346903879604562 | leider | 1.0017786861976063 | helpt | 0.9046081268620453 | kleedgeld | 1.5588902321464466 | 
 | hoeveelheden | 1.930452001280508 | telecom | 0.9064642091914905 | zonnestroom | 0.7415464892478193 | bedreigde | 0.30255954304494675 | terlet | 0.3896423179577661 | doorberekenen | 0.7542116195739694 | grootschalige | 0.4346903879604562 | voortgang | 1.0017786861976063 | mondjesmaat | 0.9046081268620453 | gedaald | 1.5588902321464466 | 
 | dierenbevrijdingsfront | 1.930452001280508 | ongevraagd | 0.9064642091914905 | centrales | 0.7415464892478193 | valkenburg | 0.30255954304494675 | zweefvliegcentrum | 0.3896423179577661 | kostenstijging | 0.7542116195739694 | vestigingsbeleid | 0.4346903879604562 | rusland | 1.0017786861976063 | reïntegreren | 0.9046081268620453 | inkomensachteruitgang | 1.5588902321464466 | 


## 6.
Evaluate your classifiers using Precision, Recall and F1. (Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes)
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.

In [82]:
def precision():
    raise NotImplemented
    
def recall():
    raise NotImplemented
    
def F1_measure():
    P = precision()
    R = recall()
    return 2*P*R/(P+R)

In [83]:
def getstats():
    ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None)

## 7 
You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.


In [84]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
    

def make_bunch(kvrdf, classes):
    """ 
    Constructs a sklearn Bunch object from a pandas dataframe. A Bunch is an Object similar to a Python dictionary
    that provides attribute-style access. (dataset["target"] == dataset.target)
    Input: pandas df, list of class names
    """

    kvrdf_np = kvrdf.values
    class_names = list(kvrdf_np[:][:, 5])
    data = list(kvrdf_np[:][:,3])
    classes = list(classes)

    target = []
    x = 0
    for name in class_names:
        target.append(classes.index(name)) 

    
    dataset = datasets.base.Bunch(data=data, target=target, target_names=classes)
    return(dataset)
    
kvrdf_train, kvrdf_test = train_test_split(kvrdf, test_size=0.2)
dataset = make_bunch(kvrdf_train, classes)
test_dataset = make_bunch(kvrdf_test, classes)

Tokenizing text files with scikit-learn

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

# Build a dictionary of features and transforms documents to feature vectors
# Absolute word count
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset.data)
X_train_counts.shape

# CALCULATE TF AND IDF
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(669, 11461)

In [86]:
#feature SELECTION
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
iris = load_iris()
X, y = X_train_tfidf, dataset.target
X.shape
selectkbest = SelectKBest(chi2, k=5000)
X_new = selectkbest.fit_transform(X, y)
X_new.shape

(669, 5000)

In [87]:
# TRAINING A CLASSIFIER
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_new, dataset.target)

In [88]:
import numpy as np
# Trying classifier on test data
X_new_counts = count_vect.transform(test_dataset.data)

X_new_tfidf = tfidf_transformer.transform(X_new_counts)

X_selected_f = selectkbest.transform(X_new_tfidf)


predicted = clf.predict(X_selected_f)

print(np.mean(predicted == test_dataset.target))

# for doc, category in zip(docs_new, predicted):
#     print('%r => %s' % (doc, dataset.target_names[category]))
#     print("\n")   

0.32142857142857145


## 8

Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>



<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  